# Logistic Regression Project

In [ ]:
data = spark.read.csv('customer_churn.csv',inferSchema=True,
                     header=True)

In [ ]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



### Check out the data

In [ ]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [ ]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

### Format for MLlib

We'll ues the numerical columns. We'll include Account Manager because its easy enough, but keep in mind it probably won't be any sort of a signal because the agency mentioned its randomly assigned!

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [ ]:
output = assembler.transform(data)

In [ ]:
final_data = output.select('features','churn')

### Test Train Split

In [ ]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

### Fit the model

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
lr_churn = LogisticRegression(labelCol='churn')

In [ ]:
fitted_churn_model = lr_churn.fit(train_churn)

In [ ]:
training_sum = fitted_churn_model.summary

In [ ]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                644|                644|
|   mean|0.17391304347826086|0.14906832298136646|
| stddev|0.37932931549818505|0.35643264681651043|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



### Evaluate results

Let's evaluate the results on the data set we were given (using the test data)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [ ]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[5.05821533910152...|[0.99368326054877...|       0.0|
|[25.0,9672.03,0.0...|    0|[5.32505105117425...|[0.99515548112882...|       0.0|
|[28.0,8670.98,0.0...|    0|[8.84581955216830...|[0.99985603833717...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.51916286796410...|[0.82041517556608...|       0.0|
|[28.0,11128.95,1....|    0|[4.50701303462802...|[0.98908900185447...|       0.0|
|[28.0,11204.23,0....|    0|[2.07039701884599...|[0.88799245575681...|       0.0|
|[29.0,9617.59,0.0...|    0|[5.02381060246666...|[0.99346359827586...|       0.0|
|[29.0,10203.18,1....|    0|[4.04435012318801...|[0.98278061484885...|       0.0|
|[30.0,6744.87,0.0...|    0|[3.99687309739191...|[0.98195847715270...|       0.0|
|[30.0,8677.28,1

### Using AUC

In [ ]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [ ]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [ ]:
auc

0.7356349589570256

[Common question - what is a good AUC value?](https://stats.stackexchange.com/questions/113326/what-is-a-good-auc-for-a-precision-recall-curve)

### Predict on brand new unlabeled data

We still need to evaluate the new_customers.csv file!

In [ ]:
final_lr_model = lr_churn.fit(final_data)

In [ ]:
new_customers = spark.read.csv('data/new_customers.csv',inferSchema=True,
                              header=True)

In [ ]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [ ]:
test_new_customers = assembler.transform(new_customers)

In [ ]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [ ]:
final_results = final_lr_model.transform(test_new_customers)

In [ ]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

